# Vertical slit-screen phase space measurement

## Setup 

In [ ]:
import sys
import os
from os.path import join
import numpy as np
import pandas as pd
import h5py
import imageio
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import cm
from matplotlib.animation import FuncAnimation
import proplot as pplt

sys.path.append('..')
from tools import energyVS06 as energy
from tools import utils
from tools import image_processing as ip

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

In [ ]:
datadir = '../Diagnostics/Data/Measurements/2022-06-26/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220626134206-y1d-vs06.h5'

In [ ]:
cam = 'cam06'
cam_settings = ip.camera_settings(cam)
zoom = 0.33
pix2mm = cam_settings.pix2mm / zoom
print(f'{cam}: {pix2mm} mm/pixel at {zoom}X zoom')

In [ ]:
# HZ34a scan
L = 8.0036 - 6.13006  # [m]

## HZ34b scan
# L = 8.0036 - 6.90450  # [m]

print(f'L = {L:.3f}')

# Slit-slit scan, L = 0.775
cam_prof_y = f'{cam}_ProfileY'
cam_image = f'{cam}_Image'
pix2mm = 0.05  # cam34 at zoom x1
ny, nx = cam_settings.shape
print('Image shape:', cam_settings.shape)

In [ ]:
file = h5py.File(join(datadir, filename), 'r')

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['/scandata']
attrs = data.dtype.names
acts = [key for key in attrs if 'PositionSync' in key]
print('attrs:', attrs)
print('acts:', acts)

Extract the vertical beam profile.

In [ ]:
signal = np.sum(data[:, cam_prof_y], axis=1)

Determine the scan center and width.

In [ ]:
y0 = 13.5
dy = 25.0

tdata = np.linspace(0.0, 1.0, len(data))

fig, ax = pplt.subplots(figsize=(4, 2))
ax.plot(tdata, data[:, acts[0]], marker='.', color='black') 
ax.format(xlabel='Time', ylabel=acts[0])
plt.show()

fig, ax = pplt.subplots(nrows=1, figsize=(4, 2))
ax.plot(data[:, acts[0]], signal, marker='.', color='black')
ax.axvline(y0 - 0.5 * dy, color='grey')
ax.axvline(y0 + 0.5 * dy, color='grey')
ax.axvline(y0, color='grey')
ax.axvspan(y0 - 0.5 * dy, y0 + 0.5 * dy, color='grey', alpha=0.2)
imax = np.argmax(signal)
ax.plot(data[imax, acts[0]], signal[imax], marker='*', color='red')
ax.format(xlabel=acts[0], ylabel=cam_prof_y)
xmin, xmax = ax.get_xlim()
if xmax < xmin:
    xmin, xmax = xmax, xmin
    ax.format(xlim=(xmin, xmax))
# plt.savefig('_output/yprof')
plt.show()

fig, ax = pplt.subplots(nrows=1, figsize=(4, 2))
ax.plot(data[:, acts[0]], signal, marker='.', color='black')
ax.axvline(y0 - 0.5 * dy, color='grey')
ax.axvline(y0 + 0.5 * dy, color='grey')
ax.axvline(y0, color='grey')
ax.axvspan(y0 - 0.5 * dy, y0 + 0.5 * dy, color='grey', alpha=0.2)
ax.format(xlabel=acts[0], ylabel=cam_prof_y, yscale='log')
plt.show()

print(f'Peak at {acts[0]} = {data[imax, acts[0]]:.3f}')

## Plot images

In [ ]:
import importlib
importlib.reload(ip)

cam_settings = ip.camera_settings(cam)
image_shape = cam_settings.shape

In [ ]:
skip = 2
nfigs = len(data[::skip])
ncols = 6
nrows = int(np.ceil(nfigs / ncols))

fig, axes = pplt.subplots(nrows=nrows, ncols=ncols, figwidth=10.0)
for i in range(nfigs):
    ax = axes[i]
    ima = np.reshape(data[::skip, cam_image][i], image_shape)
    ax.pcolormesh(np.log10(ima))
    ax.annotate(f'image {i * skip}', xy=(0.01, 0.99), xycoords='axes fraction', 
                color='white', fontsize='small',
                horizontalalignment='left', verticalalignment='top')
axes.format(xticks=[], yticks=[])
plt.show()

In [ ]:
acts

In [ ]:
for i in range(14, 21, 1):
    im = data[i, cam_image].reshape(ny, nx)

    fig, ax = pplt.subplots()
    ax.imshow(im / np.max(im), norm='log', colorbar=True)
    ax.annotate(f'frame {i}', xy=(0.02, 0.98), xycoords='axes fraction', verticalalignment='top', color='white')
    ax.annotate(f'HZ34a_PositionSync = {data[acts[0]][i]:.2f}', xy=(0.02, 0.91), 
                xycoords='axes fraction', verticalalignment='top', color='white')
    ax.format(xticks=[], yticks=[])
#     plt.savefig(f'_output/im{i}.png')
    plt.show()

## Reconstruct phase space distribution

In [ ]:
y3y1 = np.zeros([ny, len(data)])
for i in range(0, len(data)):
    im = np.reshape(data[i, cam_image], [ny, nx])
    y3y1[:, i] = np.sum(im, axis=1)
    
fig, axes = pplt.subplots(ncols=2, sharey=True)
axes[0].pcolormesh(data[:, 'y_PositionSync'], np.arange(ny), y3y1,
                   colorbar=True, colorbar_kw=dict(label=cam_image))
axes[1].pcolormesh(data[:, 'y_PositionSync'], np.arange(ny), np.log10(y3y1), 
                   colorbar=True, colorbar_kw=dict(label=f'log({cam_image})'))
axes.format(xlabel='y1', ylabel='y3')
plt.show()

In [ ]:
yscreen = pix2mm * np.arange(ny)  
Y1, Y3 = np.meshgrid(data[:, 'y_PositionSync'], yscreen)
Y = Y1 - np.average(data[:, 'y_PositionSync'], weights=np.sum(y3y1, axis=0)) 
YP = (Y3 - np.average(yscreen, weights=np.sum(y3y1, axis=1)) - Y) / L

fig, axes = pplt.subplots(ncols=2, sharey=True)
axes[0].pcolormesh(Y1, Y3, y3y1, colorbar=True)
axes[1].pcolormesh(Y1, Y3, np.log10(y3y1), colorbar=True)
axes.format(xlabel="y1", ylabel="y3")
axes[1].set_title('Logscale')
plt.show()

## Reconstruct phase space; energy slice

The y-y' phase space is already a slice in x-x' (near the beam center). The `im3d` axes are [y3, x3, y1].

In [ ]:
im3d = np.zeros([ny, nx, len(data)])  # [y_screen, x_screen, y_slit]
for i in range(0, len(data)):
    im3d[:, :, i] = np.reshape(data[i, cam_image], (ny, nx))

## Movies

#### Cam34 image vs y-slit

In [ ]:
cmap = pplt.Colormap('mono_r', discrete=False)

In [ ]:
norm_pixel_value = np.max(data[:, cam_image])
images = []
greyscale = False
colormap = cmap
for i in range(len(data)):
    im = im3d[:, :, i]  
    im = im / norm_pixel_value
    if greyscale:
        images.append(np.uint8(im * np.iinfo(np.uint8).max))
    else:
        im_rgb = colormap(im)
        images.append(np.uint8(im_rgb * np.iinfo(np.uint8).max))
    
imageio.mimsave('_output/' + filename + f'-{cam}-screen-unorm.gif', images, fps=3)
fig, ax = pplt.subplots()
ax.imshow(images[len(data) // 2], colorbar=True, cmap=cmap);

#### Slit/screen coordinates

In [ ]:
# Convert pixels to mm
x3 = np.arange(nx) * pix2mm
y3 = np.arange(ny) * pix2mm
y1 = data[:, 'y_PositionSync']

# Subtract the mean from each dimension. (Negative signs 
# flip y-slit and y-screen axes.)
x3 = x3 - np.average(x3, weights=utils.project(im3d, 1))
y3 = y3 - np.average(y3, weights=utils.project(im3d, 0))  # negative sign?
y1 = y1 - np.average(y1, weights=utils.project(im3d, 2))  # negative sign?

Y3, X3, Y1 = np.meshgrid(y3, x3, y1, indexing='ij')

Two-dimensional projections

In [ ]:
norm = None  # ['log', None]
xdata = [x3, x3, y1]
ydata = [y1, y3, y3]
Hdata = [
    utils.project(im3d, [1, 2]).T, 
    utils.project(im3d, [0, 1]), 
    utils.project(im3d, [0, 2]),
]
xlabels = ['x-screen [mm]', 'x-screen [mm]', 'y_slit [mm]']
ylabels = ['y-slit [mm]', 'y-screen [mm]', 'y-screen [mm]']
titles = ['y1-dE', 'y2-dE', 'y2-y1']
fig, axes = pplt.subplots(ncols=3, sharex=False, sharey=False, figwidth=7.5)
for x, y, H, ax, xlabel, ylabel, title in zip(xdata, ydata, Hdata, axes, xlabels, ylabels, titles):
    ax.pcolormesh(x, y, H, norm=norm)
    ax.format(xlabel=xlabel, ylabel=ylabel, title=title)

In [ ]:
# -- matrix params
a2mm = 1.009  # assume same as first dipole
rho = 0.3556
GL05 = 0
GL06 = 0.0
l1 = 0
l2 = 0
l3 = 0.774
L2 = 0.311  # slit2 to dipole face
l = 0.129  # dipole face to VS06 screen (assume same for 1st and last dipole/screen)
LL = l1 + l2 + l3 + L2  # distance from emittance plane to dipole entrance

ecalc = energy.EnergyCalculate(l1=l1, l2=l2, l3=l3, L2=L2, l=l, amp2meter=a2mm*1e3)

#Mslit = ecalc.getM1()  # slit-slit
Mscreen = ecalc.getM2()  # 2nd-slit to screen

# Matrix multiply to get 3D grids for yp and w.
Y = Y1.copy()  # [y', w, y]
YP = ecalc.calculate_yp(Y * 1e-3, Y3 * 1e-3, Mscreen)
YP = YP.reshape(Y.shape) * 1e3
W = ecalc.calculate_dE_screen(X3 * 1e-3, 0, 0, 0, Mscreen)
W = W.reshape(Y.shape) * 1e3

w = ecalc.calculate_dE_screen(x3 * 1e-3, 0, 0, 0, Mscreen) * 1e3   # energy spread

In [ ]:
from ipywidgets import interactive
from tools import plotting as mplt

def update(k):
    fig, ax = pplt.subplots()
    mplt.plot_image(im3d[:, :, k].T, norm=None, ax=ax, colorbar=True)
    plt.show()
    
interactive(update, k=(0, im3d.shape[2]))

In [ ]:
k = im3d.shape[2] // 2 + 10

_U = Y[:, :, k]
_V = YP[:, :, k]
H = im3d[:, :, k]

fig, ax = pplt.subplots()
ax.pcolormesh(H.T)
plt.show()